# Analysis of `alarm_violations.csv`

For each parameter ...
* Concerning alarm violations ...
  * Create a boxplot and stripplot based on the VALUENUM
  * Clean the data if necessary
  * Create histogram for cleaned VALUENUM  
* Concerning alarm thresholds ...
  * Create a boxplot and stripplot based on the THRESHOLD_VALUE (stratified by THRESHOLD_TYPE)
  * Clean the data if necessary
  * Create histogram for cleaned THRESHOLD_VALUE (stratified by THRESHOLD_TYPE)


## Import Data

In [ ]:
# Overview: Import all libraries used.
import numpy as np
import pandas as pd
#import scipy
#import matplotlib.pyplot as plt
import seaborn as sns
#import sklearn

In [ ]:
import pandas as pd
alarm_violations = pd.read_csv('./data/alarm_violations.csv')
alarm_violations.head()

## Parameter: O2 saturation pulseoxymetry

* `220277` **O2 saturation pulseoxymetry**, in %
* `223770` **O2 Saturation Pulseoxymetry Alarm** - Low , in %
* `223769` **O2 Saturation Pulseoxymetry Alarm** - High , in %

### O2 saturation - Alarm Violations

In [ ]:
O2_violations = alarm_violations[(alarm_violations["ITEMID"] == 220277) ]
display(O2_violations)

In [ ]:
pd.set_option('float_format', '{:f}'.format)

In [ ]:
O2_violations.VALUENUM.describe()



In [ ]:
import seaborn as sns
sns.boxplot(data=O2_violations, x='VALUENUM')

There are at least three outliers that distort the analysis. As O2 saturation is measured in %, a value above 100 is not realistic. Tat confirms our IQR, 75% of the values are below 91%.

General Introduction into O2 saturation:
* Oxygen saturation measures how much hemoglobin is currently bound to oxygen compared to how much hemoglobin remains unbound. (https://www.ncbi.nlm.nih.gov/books/NBK525974/)

Let's check the literature to see what O2 saturations can be expected based on medical knowledge.

* According to the current recommendations for post-cardiac arrest management, *oxygen therapy should only be administered in patients with oxygen saturation levels of <94% (target: oxygen saturation levels of 94–98%)* (32) (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6111205/)

https://www.ncbi.nlm.nih.gov/books/NBK525974/
* We see that we have a very small range of common values (94-98%) - everything below 94% requires an intervention

* The oxygen-hemoglobin dissociation curve is a plot of percent saturation of hemoglobin as a function of the partial pressure of oxygen (PO2). *At a PO2 of 100 mmHg, hemoglobin will be 100% saturated with oxygen*, meaning all four heme groups are bound.  - A 100% saturation is the maximum

* In the lungs, alveolar gas has a PO2 of 100 mmHg. However, due to the high affinity for the fourth oxygen molecule, oxygen saturation will remain high even at a PO2 of 60 mmHg. As the PO2 decreases, hemoglobin saturation will eventually fall rapidly, at a PO2 of 40 mmHg hemoglobin is *75% saturated*. Meanwhile, at a PO2 of 25 mmHg, hemoglobin is *50% saturated*. This level is referred to as P50, where 50% of heme groups of each hemoglobin have a molecule of oxygen bound.

* The presence of central cyanosis, blue coloration of the tongue and mucous membranes, is the most reliable predictor; it occurs at an oxyhemoglobin saturation of about 75%.

* There is no set standard of oxygen saturation where hypoxemia occurs. The generally accepted standard is that a normal resting oxygen saturation of less than 95% is considered abnormal.

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4342789/
* An SpO2 cut-off value of ≤92% is used when screening for respiratory failure in COPD patients [7]. In emergency medicine, low SpO2 has been shown to be associated with increased mortality [8,9] and is included together with other vital signs when calculating the risk score for predicting prognosis [10-13]. Different risk-scoring models to predict mortality use different limits from <90 to ≤95% [10-14].



Decision for now: 
Consider O2 Saturation above 100% as implausible values to be removed before further analysis.
No clear indication about lower boarder, but values below 50% seem very unlikely. For a broader analysis we will keep the lower boarder at 0%.

### O2 saturation - Alarm Thresholds

In [ ]:
display(O2_violations)

In [ ]:
# Check O2 saturation Alarm - High threshold
O2_violations[(O2_violations["THRESHOLD_TYPE"] == "HIGH")].THRESHOLD_VALUE.describe()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("whitegrid")

fig, (fig_box, fig_hist) = plt.subplots(2, sharex=True, gridspec_kw={"height_ratios": (.15, .85)}, figsize=(7, 5))
fig.suptitle("O2 saturation alarm thresholds of type HIGH", fontsize=16)
sns.boxplot(data=O2_violations[(O2_violations["THRESHOLD_TYPE"] == "HIGH")], x="THRESHOLD_VALUE", ax=fig_box)
fig_box.set(xlabel="")
sns.histplot(data=O2_violations[(O2_violations["THRESHOLD_TYPE"] == "HIGH")], x="THRESHOLD_VALUE", kde=True, ax=fig_hist)
fig_hist.set_xlabel("THRESHOLD_VALUE (O2 saturation in %)", fontsize=12)
fig_hist.set_ylabel("Count", fontsize=12)

plt.show()

The minimum value of the HIGH alarm thresholds is 0, which is surprisingly low.

Let's check the suspiciously low HIGH alarm thresholds.

In [ ]:
# Looking at THRESHOLD_VALUE (high) smaller than 90 ...
O2_threshold_check_high = O2_violations[(O2_violations["THRESHOLD_TYPE"] == "HIGH") & (O2_violations["THRESHOLD_VALUE"] < 90)].sort_values(by=['THRESHOLD_VALUE'])
display(O2_threshold_check_high)
O2_threshold_check_high.THRESHOLD_VALUE.describe()

In [ ]:
sns.boxplot(data=O2_threshold_check_high, x='THRESHOLD_VALUE')

There are 5649 THRESHOLD_VALUEs that are 0 >= x < 90 for thresholds of type HIGH, which is suspicious.

 A possible explanation could be that ICU staff sets a too low threshold by mistake (e.g. typing 0 instead of 100). This would directly trigger an alarm or wanted to set it as low Threshold Type, not high.

Keeping in mind that the data set includes only the violated thresholds; unusually high thresholds will rarely trigger an alarm, while unusually low alarms (such as 0 bpm) will immediately cause an alarm in living people.

In [ ]:
# Check O2 saturation Alarm - Low threshold
O2_violations[(O2_violations["THRESHOLD_TYPE"] == "LOW")].THRESHOLD_VALUE.describe()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("whitegrid")

fig = sns.stripplot(data=O2_violations[(O2_violations["THRESHOLD_TYPE"] == "LOW")], x="THRESHOLD_VALUE")
fig.set_title("Scatterplot for O2 saturation alarm thresholds of type LOW", fontsize=14)
fig.set_xlabel("THRESHOLD_VALUE (O2 saturation in %)", fontsize=12)

plt.show(fig)

The maximum value of the LOW alarm thresholds is 90100, which is too high and might again indicate a decimal issue (90.10?).

Let's check the suspiciously high LOW alarm thresholds.

In [ ]:
O2_threshold_check_low = O2_violations[(O2_violations["THRESHOLD_TYPE"] == "LOW") & (O2_violations["THRESHOLD_VALUE"] > 99)].sort_values(by=['THRESHOLD_VALUE'])
display(O2_threshold_check_low)
O2_threshold_check_low.THRESHOLD_VALUE.describe()

In [ ]:
O2_threshold_check_low_01 = O2_violations[(O2_violations["THRESHOLD_TYPE"] == "LOW") & (O2_violations["THRESHOLD_VALUE"] > 99) & (O2_violations["THRESHOLD_VALUE"] <= 200)].sort_values(by=['THRESHOLD_VALUE'])
O2_threshold_check_low_02 = O2_violations[(O2_violations["THRESHOLD_TYPE"] == "LOW") & (O2_violations["THRESHOLD_VALUE"] > 200) & (O2_violations["THRESHOLD_VALUE"] <= 2000)].sort_values(by=['THRESHOLD_VALUE'])
O2_threshold_check_low_03 = O2_violations[(O2_violations["THRESHOLD_TYPE"] == "LOW") & (O2_violations["THRESHOLD_VALUE"] > 2000)].sort_values(by=['THRESHOLD_VALUE'])



import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
fig, axs = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle("A closer look at the suspiciously high LOW thresholds", fontsize=16)

sns.stripplot(ax=axs[0], data=O2_threshold_check_low_01, x="THRESHOLD_VALUE")
axs[0].set_title("O2 alarm thresholds LOW 99>x<=200", fontsize=14)
axs[0].set_xlabel("THRESHOLD_VALUE (O2 saturation in %)", fontsize=12)

sns.stripplot(ax=axs[1], data=O2_threshold_check_low_02, x="THRESHOLD_VALUE")
axs[1].set_title("O2 alarm thresholds LOW 200>x<=2000", fontsize=14)
axs[1].set_xlabel("THRESHOLD_VALUE (O2 saturation in %)", fontsize=12)
axs[1].set_xlabel("THRESHOLD_VALUE (O2 saturation in %)", fontsize=12)

sns.stripplot(ax=axs[2], data=O2_threshold_check_low_03, x="THRESHOLD_VALUE")
axs[2].set_title("O2 alarm thresholds LOW x>2000", fontsize=14)
axs[2].set_xlabel("THRESHOLD_VALUE (O2 saturation in %)", fontsize=12)

plt.show(fig)

In [ ]:
O2_threshold_check_low_01.THRESHOLD_VALUE.describe()

In [ ]:
O2_threshold_check_low_02.THRESHOLD_VALUE.describe()

In [ ]:
O2_threshold_check_low_03.THRESHOLD_VALUE.describe()

## Inclusion Criteria

### For VALUENUM 
For O2 saturation Values the inclusion interval looks as follows: 0>=x<=100 (not so strict version) - 50>=x<=100 (strict version)
* Lower Boarder: 0% (even though values below 50% seem very unlikely)
* Upper Boarder: 100%

### For THRESHOLD_VALUE
For Thresholds of type **HIGH** the inclusion interval looks as follows: 90>=x<=100 (not so strict version) - 98>0x<=100 (strict version)
* HIGH Threshold Values below 90 (even though below 94 seems unlikely) get excluded.
* HIGH Threshold Values above 100 get excluded (There are now HIGH Thresholds above 100 so this does not have an impact on the existing data)
* *A HIGH Threshold for O2 saturation in general does not make much sense but as long as the O2 saturation is not 100% it is should not be a mdeical problem, so one could also propose that the high thresholds need to be between 99 and 100.*

For Thresholds of type **LOW** the inclusion interval looks as follows: 0>=x<99.5 (not so strict version) - 50>=x<=94 (strict version)
* LOW Thresholds can be set till 0 (even though an alarm for low should at least arrive at 80% - mybe 50% for ICU)
* LOW Thresholds can be set till 99.5 (even though medical implications only arise after 94%)
* *A LOW Threshold for O2 saturation is more important, but generating an alarm before the value is below 94% might not be relevant*



### Record Losses by Inclusion Criteria


### For VALUENUM
* 0 removed records due to too low value (<0)
* 44 removed records due to too high value (>100)

In [ ]:
O2_violations_clean = O2_violations[(O2_violations["VALUENUM"] >= 0) & (O2_violations["VALUENUM"] <= 100)]
O2_violations_removed_too_low = O2_violations[(O2_violations["VALUENUM"] < 0)]
O2_violations_removed_too_high = O2_violations[(O2_violations["VALUENUM"] > 100)]

In [ ]:
# Check rows that were removed because of too low VALUENUM
display(O2_violations_removed_too_low.sort_values(by=['VALUENUM']))
O2_violations_removed_too_low.VALUENUM.describe()
# No values were removed due to lower boarder 

In [ ]:
# Check rows that were removed because of too high VALUENUM
display(O2_violations_removed_too_high.sort_values(by=['VALUENUM']))
O2_violations_removed_too_high.VALUENUM.describe()
# 44 rows were removed due to too high values for O2 saturation
# They range from 110 to ~6mio. Many of the are in the houndreds & thousand beginning with 9 - one could assume a decimal issue?

### For THRESHOLD VALUES
HIGH:
* 5649 rows removed due to high threshold values below 90 (ranging from 0 to 89)
* 0 rows removed due to high threshold values above 100

LOW:
* 0 rows removed due to low threshold values below 0
* 1999 rows removed due to high threshold values above 99.5 (ranging from 100 to 90100)

In [ ]:
O2_violations_clean_threshold_high = O2_violations[(O2_violations["THRESHOLD_TYPE"] == 'HIGH') & (O2_violations["THRESHOLD_VALUE"] >= 90) & (O2_violations["THRESHOLD_VALUE"] <= 100)]
O2_violations_removed_threshold_high_too_low = O2_violations[(O2_violations["THRESHOLD_TYPE"] == 'HIGH') & (O2_violations["THRESHOLD_VALUE"] < 90)]
O2_violations_removed_threshold_high_too_high = O2_violations[(O2_violations["THRESHOLD_TYPE"] == 'HIGH') & (O2_violations["THRESHOLD_VALUE"] > 100)]


In [ ]:
O2_violations_clean_threshold_high.describe()

In [ ]:
# Check rows that were removed because of too low THRESHOLD_VALUE for Type HIGH
display(O2_violations_removed_threshold_high_too_low.sort_values(by=['THRESHOLD_VALUE']))
O2_violations_removed_threshold_high_too_low.THRESHOLD_VALUE.describe()
# 5649 rows removed

In [ ]:
# Check rows that were removed because of too high THRESHOLD_VALUE for Type HIGH
display(O2_violations_removed_threshold_high_too_high.sort_values(by=['THRESHOLD_VALUE']))
O2_violations_removed_threshold_high_too_high.THRESHOLD_VALUE.describe()
# 0 rows removed

In [ ]:
O2_violations_clean_threshold_low = O2_violations[(O2_violations["THRESHOLD_TYPE"] == 'LOW') & (O2_violations["THRESHOLD_VALUE"] >= 0) & (O2_violations["THRESHOLD_VALUE"] <= 99.5)]
O2_violations_removed_threshold_low_too_low = O2_violations[(O2_violations["THRESHOLD_TYPE"] == 'LOW') & (O2_violations["THRESHOLD_VALUE"] < 0)]
O2_violations_removed_threshold_low_too_high = O2_violations[(O2_violations["THRESHOLD_TYPE"] == 'LOW') & (O2_violations["THRESHOLD_VALUE"] > 99.5)]


In [ ]:
# Check rows that were removed because of too low THRESHOLD_VALUE for Type LOW
display(O2_violations_removed_threshold_low_too_low.sort_values(by=['THRESHOLD_VALUE']))
O2_violations_removed_threshold_low_too_low.THRESHOLD_VALUE.describe()
# 0 rows removed

In [ ]:
# Check rows that were removed because of too high THRESHOLD_VALUE for Type LOW
display(O2_violations_removed_threshold_low_too_high.sort_values(by=['THRESHOLD_VALUE']))
O2_violations_removed_threshold_low_too_high.THRESHOLD_VALUE.describe()
# 1999 rows removed (rangeing fromm 100 to 90100)

## Cleaned Data Set


### Cleaned Values

In [ ]:
# Check cleaned value O2_violations
O2_violations_clean_value = O2_violations[(O2_violations["VALUENUM"] >= 0) & (O2_violations["VALUENUM"] <= 100)]
display(O2_violations_clean_value.sort_values(by=['VALUENUM']))
O2_violations_clean_value.VALUENUM.describe()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

fig, axs = plt.subplots(1, 3, figsize=(25, 5))
fig.suptitle("O2 saturation - Alarm Violations", fontsize=18)

sns.stripplot(data=O2_violations_clean_value, x='VALUENUM', ax=axs[0])
axs[0].set_title("O2_violations_clean_value scatter plot")
axs[0].set_xlabel("O2_violations_clean VALUENUM")

sns.boxplot(data=O2_violations_clean_value, x='VALUENUM', ax=axs[1])
axs[1].set_title("O2_violations_clean_value boxplot")
axs[1].set_xlabel("O2_violations_clean VALUENUM")

sns.histplot(data=O2_violations_clean_value, x='VALUENUM', ax=axs[2])
axs[2].set_title("O2_violations_clean_value histogram")
axs[2].set_xlabel("O2_violations_clean VALUENUM")

plt.show(fig)
# We still have many values that seem to be outliers following common O2 saturations (below 70%)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("whitegrid")

fig, (fig_box, fig_hist) = plt.subplots(2, sharex=True, gridspec_kw={"height_ratios": (.15, .85)}, figsize=(7, 5))
fig.suptitle("O2 saturation alarm violations in cleaned data set", fontsize=16)
sns.boxplot(data=O2_violations_clean_value, x="VALUENUM", ax=fig_box)
fig_box.set(xlabel="")
sns.histplot(data=O2_violations_clean_value, x="VALUENUM", kde=True, ax=fig_hist)
fig_hist.set_xlabel("VALUENUM (O2 saturation in %)", fontsize=12)
fig_hist.set_ylabel("Count", fontsize=12)

plt.show()

Cleaned Values + Cleaned HIGH Thresholds


In [ ]:
O2_violations_clean_threshold_high = O2_violations_clean_value[(O2_violations_clean_value["THRESHOLD_TYPE"] == 'HIGH') & (O2_violations_clean_value["THRESHOLD_VALUE"] >= 90) & (O2_violations_clean_value["THRESHOLD_VALUE"] <= 100)]

In [ ]:
O2_violations_clean_threshold_high.describe()


In [ ]:
# The rows with THRESHOLD_VALUE 100 for Type high where somehow removed. Check what their values were
O2_violations[(O2_violations["THRESHOLD_TYPE"] == "HIGH") & (O2_violations["THRESHOLD_VALUE"] == 100) ]
# weirdly all record with Threshold value 100 have too high values (above 100) and were therefore filtered out in the previous step



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("whitegrid")

fig, (fig_box, fig_hist) = plt.subplots(2, sharex=True, gridspec_kw={"height_ratios": (.15, .85)}, figsize=(7, 5))
fig.suptitle("O2 saturation alarm thresholds of type HIGH - cleaned", fontsize=16)
sns.boxplot(data=O2_violations_clean_threshold_high, x="THRESHOLD_VALUE", ax=fig_box)
fig_box.set(xlabel="")
sns.histplot(data=O2_violations_clean_threshold_high, x="THRESHOLD_VALUE", kde=True, ax=fig_hist)
fig_hist.set_xlabel("THRESHOLD_VALUE (O2 saturation in %)", fontsize=12)
fig_hist.set_ylabel("Count", fontsize=12)

plt.show()

Cleaned Values + Cleaned LOW Thresholds

In [ ]:
O2_violations_clean_threshold_low = O2_violations_clean_value[(O2_violations_clean_value["THRESHOLD_TYPE"] == 'LOW') & (O2_violations_clean_value["THRESHOLD_VALUE"] >= 0) & (O2_violations_clean_value["THRESHOLD_VALUE"] <= 99.5)]

In [ ]:
O2_violations_clean_threshold_low.describe()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("whitegrid")

fig, (fig_box, fig_hist) = plt.subplots(2, sharex=True, gridspec_kw={"height_ratios": (.15, .85)}, figsize=(7, 5))
fig.suptitle("O2 saturation alarm thresholds of type LOW - cleaned", fontsize=16)
sns.boxplot(data=O2_violations_clean_threshold_low, x="THRESHOLD_VALUE", ax=fig_box)
fig_box.set(xlabel="")
sns.histplot(data=O2_violations_clean_threshold_low, x="THRESHOLD_VALUE", kde=True, ax=fig_hist)
fig_hist.set_xlabel("THRESHOLD_VALUE (O2 saturation in %)", fontsize=12)
fig_hist.set_ylabel("Count", fontsize=12)

plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("whitegrid")

fig = sns.stripplot(data=O2_violations_clean_threshold_low, x="THRESHOLD_VALUE")
fig.set_title("Scatterplot for O2 saturation alarm thresholds of type LOW - cleaned", fontsize=14)
fig.set_xlabel("THRESHOLD_VALUE (O2 saturation in %)", fontsize=12)

plt.show(fig)

In [ ]:
frames = [O2_violations_clean_threshold_low,O2_violations_clean_threshold_high]
O2_violations_clean = pd.concat(frames)

In [ ]:
O2_violations_clean.describe()

## Additional Visualizations

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
fig, axs = plt.subplots(1, 2, figsize=(20, 5))
fig.suptitle("Distribution of O2 saturation alarm violations stratified by threshold type in cleaned data set", fontsize=18)

sns.histplot(ax=axs[0], data=O2_violations_clean, x="VALUENUM", hue="THRESHOLD_TYPE", palette=["darkblue", "darkgreen"])
axs[0].set_title("Histogram", fontsize=12)
axs[0].set_xlabel("VALUENUM (O2 saturation in %)", fontsize=12)
axs[0].set_ylabel("Count", fontsize=12)

sns.kdeplot(ax=axs[1], data=O2_violations_clean, x="VALUENUM", hue="THRESHOLD_TYPE", palette=["darkblue", "darkgreen"])
axs[1].set_title("Kernel density estimate (KDE)", fontsize=12)
axs[1].set_xlabel("VALUENUM (O2 saturation in %)", fontsize=12)
axs[1].set_ylabel("Density", fontsize=12)

plt.show(fig)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("whitegrid")

fig, (fig_box, fig_hist) = plt.subplots(2, sharex=True, gridspec_kw={"height_ratios": (.15, .85)}, figsize=(10, 5))
fig.suptitle("O2 saturation alarm violations in cleaned data set", fontsize=18)
sns.boxplot(data=O2_violations_clean, x="VALUENUM", ax=fig_box)
fig_box.set(xlabel="")
sns.histplot(data=O2_violations_clean, x="VALUENUM", kde=True, ax=fig_hist)
fig_hist.set_xlabel("VALUENUM (O2 saturation in %)", fontsize=12)
fig_hist.set_ylabel("Count", fontsize=12)

plt.show()